In [1]:
# import key libs
import os, rasterio
import numpy as np
from tqdm import tqdm
from math import ceil
import multiprocessing as mp
from scipy.interpolate import interp1d

# import clustering related modules
from cuml import UMAP
from sknetwork.clustering import Louvain
from sklearn.model_selection import train_test_split as TTS
from sklearn.preprocessing import minmax_scale, MinMaxScaler, StandardScaler

# import plotting libs
import glasbey
import matplotlib as mpl
import matplotlib.pyplot as plt


from csiro_spectral_tools.hulls.convexhulls import uc_hulls

from src import *
import torch
import torch.nn as nn
from fastai.vision.all import *
from torch.utils.data import TensorDataset, DataLoader

## Set Data Paths

In [2]:

# raw spectral data cube
hsi_fn = "*.bil"

# set path to cluster fn
cluster_fn = ''

# set path to output directory on EFS
out_dir = "/mnt/outputs/"  
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

## Generate Sampling Mask

Read the cluster map from the 2nd iteration of clustering to generate a boolean pixel selection map. This defines regions of the data cube that are unvegetated, not water and not in the shade. 

In [ ]:
%%time


if os.path.exists(cluster_fn):
    # load the cluster map
    with rasterio.open(cluster_fn, 'r') as src:
        cluster_map = src.read(1)
    # create a boolean mask
    geo_map = np.invert(np.isin(cluster_map, [999, 6, 15, 5, 25, 26]))

    # plot for sanity check
    plt.imshow(geo_map)
    plt.show()
else:
    src = rasterio.open(hsi_fn, "r")
    print(src.shape)
    print(src.res)
    arr = src.read(30)
    src.close()
    geo_map = np.invert(np.isin(arr, [-99]))
    fig, ax=plt.subplots(1, 2)
    ax[0].imshow(geo_map)
    ax[1].imshow(arr)
    plt.show()

## Sample Data Cube

Use hyperlib function to sample the data cube at a spatial resolution of 25 metres within the sampling map.

In [4]:
# use the geo map to sample the data cube
spc = 7 #m
samples = sample_parallel_Nth_pixel_with_map(hsi_fn, spacing=spc, label_map=geo_map)
samples.shape

Working on 12 processors...


100%|██████████| 3494/3494 [06:09<00:00,  9.46it/s]


(6290218, 125)

In [5]:
# get wavelengths from cube
with rasterio.open(hsi_fn, 'r') as src:
    wvl = np.array([float(x.split(' ')[-1].strip('()'))*1000 for x in src.descriptions])

# generate a rounded number in millions for the number of samples
N_samps = round(samples.shape[0] / 1000000, 2)

print(np.diff(wvl))
wvl.shape, N_samps

[ 14.8  14.7  14.7  14.7  14.6  14.7  14.6  14.7  14.5  14.6  14.6  14.5
  14.6  14.5  14.5  14.5  14.4  14.5  14.4  14.4  14.4  14.3  14.4  14.3
  14.3  14.3  14.3  14.3  14.2  14.3   3.9  15.6  15.5  15.5  15.4  15.3
  15.3  15.1  15.1  15.   14.9  14.9  14.8  14.7  14.6  14.6  14.4  14.4
  14.4  14.2  14.2  14.1  14.   13.9  13.9  13.8  13.7  13.6  13.6  13.5
  13.4  57.1  15.   15.   14.8  14.8  14.6  14.5  14.5  14.3  14.2  14.1
  13.9  13.9  13.8  13.7  13.5  13.5  13.3  13.3  13.1  13.1  12.9  12.8
  12.7  12.6  12.5  12.4  12.3  12.1  12.1  12.   11.8 133.2  19.4  19.3
  19.2  19.   18.9  18.8  18.7  18.6  18.4  18.3  18.2  18.   18.   17.8
  17.6  17.6  17.4  17.3  17.2  17.1  16.9  16.8  16.7  16.6  16.4  16.3
  16.2  16.1  15.9  15.8]


((125,), 6.29)

## Run Convex Hull 

Run convex hull on part of the spectrum unaffected by atmospheric OH absorption at ~1400nm and ~1900nm, vegetation below ~750nm and poor signal/noise above ~2420nm.

In [ ]:
# find 5 wavelengths either side of the atmospheric windows
w = np.ones(wvl.shape[0]).astype(bool)
for i in np.argwhere(np.diff(wvl) > 30).flatten():
    sp = 3 # 5
    for j in range(-sp, sp):
        w[i-j] = False

# discard 5 wavelenths at end & discard all wavelength < 750nm
w[:np.argmin(abs(wvl-750))] = False
w[-5:] = False

# visualise the spectral observation to keep and discard on random spectra
fig, axes = plt.subplots(3,2,figsize=(10, 10), constrained_layout=True)
fig.patch.set_alpha(0.0)

for i, ax in enumerate(axes.flatten()):
    idx = np.random.choice(np.arange(samples.shape[0]))
    ax.scatter(wvl[w], samples[idx,w], label='Keep')
    ax.scatter(wvl[~w], samples[idx,~w], label='Discard')
    ax.plot(wvl[w], samples[idx,w], c='k')
    ax.grid(linestyle='--')
    ax.set(title='Random Spectra', ylabel='Reflectance', xlabel='Wavelength (nm)', facecolor='#e6eaf6')
    ax.legend()
plt.show()

In [7]:
%%time
# define a function that runs parallelised convex hull continuum removal
def convex_hull_cont_remove(X, wvl):
    N = mp.cpu_count()
    in_data = [(wvl, spec) for spec in X]
    # run parallised workflow
    with mp.Pool(processes=N) as p:
        results = p.map(calc_convex_hull, in_data) # per pixel upper monotone chain residual
    return np.stack(results)

def cvx_quotient(ar, wvl):
    ar = ar.T - ar.T.min(axis=0)  # set spectral minimum value to 0
    ar = ar.T
    if len(ar.shape) == 1:
        result = -uc_hulls(wvl, ar, 0) + 1
        return result[1:-1].astype("float32")
    if len(ar.shape) == 2:
        result = -uc_hulls(wvl, ar, 0) + 1
        return result[:, 1:-1].astype("float32")
    if len(ar.shape) == 3:
        result = -uc_hulls(wvl, ar, 0) + 1
        return result[:, :, 1:-1].astype("float32")

# cvx_samples = convex_hull_cont_remove(samples[:,w], wvl[w])
cvx_samples = cvx_quotient(samples[:, w], wvl[w])

CPU times: user 23.7 s, sys: 3.99 s, total: 27.6 s
Wall time: 27.6 s


In [8]:
samples.shape

(6290218, 125)

In [ ]:
# plot some convex spectra
fig, ax = plt.subplots(figsize=(11,5))
for _ in range(1000):
    idx = np.random.choice(np.arange(samples.shape[0]))
    # ax.plot(wvl[w], cvx_samples[idx], alpha=0.02, c='k')
    ax.plot(wvl[w][1:-1], cvx_samples[idx], alpha=0.02, c='k')
plt.show()

In [ ]:
# plot some spectra scaled 0 to 1
fig, ax = plt.subplots(figsize=(11,5))
for _ in range(1000):
    idx = np.random.choice(np.arange(samples.shape[0]))
    ax.plot(wvl[w][1:-1], scale_spectra(cvx_samples[idx]), alpha=0.02, c='k')
plt.show()

## Run Clustering

In [35]:
torch.cuda.empty_cache()

In [20]:
%%time
# run 0-1 scaling on convex hull spectra
# X = scale_spectra(cvx_samples)
X = cvx_samples # if already scaled

# fit umap to scaled spectra
umap_obj = UMAP(n_components=2, min_dist=0.05, n_neighbors=23, metric='cosine')
# umap_obj = UMAP(n_components=2, min_dist=0.05, n_neighbors=20, metric='euclidean')
embeds = umap_obj.fit_transform(X)

CPU times: user 7min 20s, sys: 36 s, total: 7min 56s
Wall time: 1min 37s


In [ ]:
# plot the embeddings space
fig, ax = plt.subplots(figsize=(10,10))
fig.patch.set_alpha(0.0)
ax.scatter(embeds[:,0], embeds[:,1], c='k', s=0.05, alpha=0.05)
ax.set(title=f'2D UMAP - CVX Spectra - Restricted Wavelengths - 25m Sample (N={N_samps}M)',
        ylabel='UMAP1', xlabel='UMAP0',facecolor='#e6eaf6', aspect=1)
ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
ax.grid(linestyle='--')
plt.show()

In [22]:
# extract the graph from the umap object
graph = umap_obj.graph_.get()

# run louvain clustering
labels = Louvain().fit_predict(graph)
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [23]:
# get colour dictionary for labels, use glasbey to extend colour palette
colormap = plt.cm.get_cmap('Dark2', len(np.unique(labels)))
colordict = dict(zip(sorted(np.unique(labels)), glasbey.extend_palette('Dark2', palette_size=np.unique(labels).shape[0])))                         

/tmp/ipykernel_44949/3052606257.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = plt.cm.get_cmap('Dark2', len(np.unique(labels)))


In [ ]:
# calculate unique classes and class counts
classes, counts = np.unique(labels, return_counts=True)

# plot a bar chart
fig, ax = plt.subplots(figsize=(12,5))
fig.patch.set_alpha(0.0)
for i in classes:
    ax.bar(str(i), counts[i], color=colordict[i])
    ax.text(i, counts[i]+6000, f'{counts[i]}', rotation='vertical', ha='center')
ax.set(title=f'CVX Spectra Mineral Feature Windows - Louvain Cluster Counts', 
       facecolor='#e6eaf6', ylim=(0, counts[0]+50000), ylabel='Counts', xlabel='Cluster ID')
ax.grid(axis='y', linestyle='--')
plt.show()

In [25]:
# generate a mask for pixels containing valid clusters with >1000 classes
valid_cluster_mask = np.isin(labels, classes[np.argwhere(counts > 1000)].flatten())

In [ ]:
# plot the embeddings space with labels
fig, ax = plt.subplots(figsize=(10,10))
fig.patch.set_alpha(0.0)
for i in np.unique(labels[valid_cluster_mask]):
    N = round(labels[labels==i].shape[0]/1000, 2)
    ax.scatter(embeds[labels==i,0], embeds[labels==i,1], c=colordict[i], s=0.1, alpha=0.8, label=f'Cluster{i} (N={N}K)', linewidth=0)

ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
ax.set(title=f'2D UMAP - CVX Spectra Mineral Feature Windows - 25m Sample (N={N_samps}M) - Clustered', 
       ylabel='UMAP1', xlabel='UMAP0',facecolor='#e6eaf6', aspect=1)
ax.grid(linestyle='--')
ax.legend(title='Louvain Labels', loc=(1.02,0.01), markerscale=20)
plt.show()

In [28]:
# save data for model training
out_fn = os.path.join(out_dir, 'Peru_D_Refl_Crop_refl_7m_cvxq.hdf5')
write_hdf5_datafile(
    out_fn = out_fn, 
    data_dict = {
        'raw_spectra' : {'raw':samples[valid_cluster_mask], 'wvl':wvl},
        'cvx_spectra' : {'cvx_samples':cvx_samples[valid_cluster_mask], 'cvx_wvl':wvl[w]},
        'embeds' : {'embeds':embeds[valid_cluster_mask]},
        'labels' : {'labels':labels[valid_cluster_mask]},
    }
)

In [ ]:
# plot cluster convex hull spectra
for j in np.unique(labels[valid_cluster_mask]):
    fig, ax = plt.subplots(figsize=(10,3))
    for _ in range(1000):
        idx = np.random.choice(np.argwhere(labels==j).flatten())
        ax.plot(wvl[w][1:-1], cvx_samples[idx], alpha=0.02, c='k')
    ax.plot(wvl[w][1:-1], cvx_samples[labels==j].mean(axis=0), c='r')
    ax.set(title=f'Cluster {j}')
    plt.show()

## Train Classifier and Run Inference

Use cluster labels on spatial sample to train a supervised classifier to infer on the data cube. This uses the raw spectral information as model input to speed up inference on the raw data cube.

In [31]:
samples[1]

array([ 311.8632,  513.6846,  649.366 ,  772.5616,  907.1935,  994.1978,
       1051.2909, 1096.2058, 1105.4993, 1162.3411, 1237.4094, 1322.6422,
       1409.172 , 1485.931 , 1562.9373, 1690.3297, 1945.7532, 2295.3774,
       2614.3958, 2833.9604, 3012.3777, 3134.6343, 3229.6843, 3336.366 ,
       3446.9905, 3512.6748, 3613.3767, 3661.7424, 3807.4878, 3811.5317,
       3937.0503, 4006.7273, 4124.157 , 4182.3535, 4255.4507, 4388.5913,
       4464.8325, 4563.2744, 4667.377 , 4742.196 , 4837.2886, 4902.592 ,
       4984.2046, 5018.742 , 5060.69  , 5084.8228, 5040.178 , 5081.2046,
       5004.1587, 4996.5845, 4957.3755, 4903.4697, 4853.585 , 4824.22  ,
       4774.209 , 4728.2046, 4670.746 , 4600.6934, 4525.366 , 4429.598 ,
       4327.56  , 4217.0273, 2544.7024, 2952.3467, 2908.2434, 2815.0044,
       2890.9458, 2903.1333, 3000.3838, 3136.792 , 3276.893 , 3405.0952,
       3538.6055, 3642.487 , 3757.1895, 3874.2764, 3995.36  , 4120.2827,
       4242.9507, 4346.622 , 4413.857 , 4456.455 , 

In [32]:
# create training and testing subsets of the raw sample
# X_train, X_test, y_train, y_test = TTS(scale_spectra(samples[valid_cluster_mask]), labels[valid_cluster_mask], test_size=0.3)
X_train, X_test, y_train, y_test = TTS(scale_spectra(samples[valid_cluster_mask]), labels[valid_cluster_mask], test_size=0.3)

# create tensors
X_train, y_train = classify_Xy_to_tensor(X_train, y_train)
X_test, y_test = classify_Xy_to_tensor(X_test, y_test)
y_test, y_test.shape, X_test.shape, y_train.shape, X_train.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([1886971, 23]),
 torch.Size([1886971, 125]),
 torch.Size([4402930, 23]),
 torch.Size([4402930, 125]))

In [33]:
# define model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = conv1d_classify_nn( 
    input_channels = X_train.shape[1],
    output_channels = y_train.shape[1]).to(device)

# build training and testing dataloaders
train_dl = DataLoader(TensorDataset(X_train, y_train), batch_size=40000, shuffle=True)
test_dl = DataLoader(TensorDataset(X_test, y_test), batch_size=40000, shuffle=True)

# set a list of callbacks to track model fitting progress
callbacks = [
    CSVLogger(fname=os.path.join(out_dir, 'area_d_cvx_swir_only.log'), append=False),
    EarlyStoppingCallback(monitor="valid_loss", min_delta=0.0005, patience=15),
]

# use the fastai learner to fit some epochs
learn = Learner(DataLoaders(train_dl, test_dl), model, loss_func=torch.nn.BCELoss(), opt_func=Adam,
                path=out_dir, model_dir='area_d_cvx_swir_only')
learn.fit(550, 0.001, cbs=callbacks)

# save the model 
learn.save(file='area_D_cvx_swir_only', with_opt=True)

epoch,train_loss,valid_loss,time


No improvement since epoch 155: early stopping


Path('/mnt/outputs/intermediate/area_d_cvx_swir_only/area_D_cvx_swir_only.pth')

In [ ]:
# plot the progres from the log
df = pd.read_csv(os.path.join(out_dir, 'area_d_cvx_swir_only.log'))
df.time = df['time'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
df.time = df.time.cumsum() / 60

fig, ax = plt.subplots(figsize=(9,4))
fig.patch.set_alpha(0.0)
ax.plot(df.time, df.train_loss, label='Training Loss')
ax.plot(df.time, df.valid_loss, label='Validation Loss')
ax.grid(linestyle='--')
ax.set(title='Multiclass Spectral Cluster Training & Validation Loss Curve', xlabel='Elapsed Time (minutes)', 
    ylabel='Binary Cross Entropy Loss', facecolor='#e6eaf6', yscale='log')
ax.legend()
plt.show()

In [35]:
# set output file name
out_fn = os.path.join(out_dir, 'area_d_cvxq_SWIR_clusters.tif')

# make a custom inference loop to write the class label, not probability
with rasterio.open(hsi_fn, 'r') as src:
    meta = src.meta.copy()
    meta.update({'count':1, 'driver':'GTIFF', 'BIGTIFF':'YES', 'nodata':999, 'dtype':'int16'})
    nodata_mask = src.read(1) == src.nodata
    with rasterio.open(out_fn, 'w', **meta) as dst:
        for ji, block in tqdm([x for x in src.block_windows()]):
            hsi_window = src.read(window=block)[:, 0, :].T
            nodata_window = nodata_mask[ji[0], :]
            hsi_window = scale_spectra(hsi_window)
            pred_window = get_1dcnn_predictions(learn, hsi_window, nodata_window, src.nodata) # get probabilities
            pred_labels = np.argmax(pred_window, axis=1).astype('int16') # argmax for class
            pred_labels[nodata_window] = 999
            dst.write(np.array([pred_labels]), 1, window=block) 

# edit nodata values  using the sampling map to mask out of bounds predictions
with rasterio.open(out_fn, 'r') as src:
    meta = src.meta.copy()
    with rasterio.open(out_fn.replace('.tif','_tmp.tif'), 'w', **meta) as dst:
        a = src.read(1)
        a[np.invert(geo_map)] = src.nodata
        dst.write(a, 1)

# remove original, rename masked file and convert to cloud optimised geotiff
os.remove(out_fn)
os.rename(out_fn.replace('.tif','_tmp.tif'), out_fn)
convert_to_cog(out_fn)

  0%|          | 0/6994 [00:00<?, ?it/s]/workspaces/tool-geoscience-appdeck/a3-hsi-clustering/src/core.py:22: RuntimeWarning: invalid value encountered in divide
  X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
100%|██████████| 6994/6994 [13:26<00:00,  8.67it/s]


0...10...20...30...40...50...60...70...80...90...100 - done.
Converting file to cloud optimised geotiff
	/mnt/outputs/intermediate/area_d_cvxq_SWIR_clusters.tif


In [ ]:

out_fn = os.path.join(out_dir, 'area_d_cvxq_SWIR_clusters.tif')
out_fn
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Open the raster file
with rasterio.open(out_fn) as src:
    fig, ax = plt.subplots(figsize=(10, 10))
    data = src.read(1)  # Read the first band

    nodata_value = 999
    data = np.ma.masked_equal(data, nodata_value)

    # Create a discrete colormap
    cmap = plt.get_cmap('tab20', np.max(data) - np.min(data) + 1)
 
    show(src, ax=ax, title='area d cvxq clusters', cmap=cmap)
    cbar = fig.colorbar(cax, ticks=np.arange(np.min(data), np.max(data) + 1))
    plt.show()

In [ ]:
import colorcet as cc
out_fn = os.path.join(out_dir, 'area_d_cvxq_SWIR_clusters.tif')
# Open the raster file
with rasterio.open(out_fn) as src:
    data = src.read(1)  # Read the first band

    nodata_value = 999
    data = np.ma.masked_equal(data, nodata_value)

    # Create a discrete colormap
    cmap = plt.get_cmap('tab20', np.max(data) - np.min(data) + 1)
    # cmap = cc.cm.glasbey_bw_minc_20_maxl_70

    fig, ax = plt.subplots(figsize=(15, 15))
    cax = ax.imshow(data, cmap=cmap, vmin=np.min(data) - 0.5, vmax=np.max(data) + 0.5)
    cbar = fig.colorbar(cax, ticks=np.arange(np.min(data), np.max(data) + 1))
    ax.set_title('area D cvxq cluster Map'.title())
    plt.show()

In [ ]:
cmap